In [15]:
from glob import glob
from os import path
import seaborn as sns
import json
import os
import numpy as np
import pandas as pd
import re
from os.path import basename, splitext
import time
import pickle
import json
from datetime import datetime

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering import run_roster_solver_objval_w_baseline

In [16]:
def load_shift(shift_file, OC, RM, GM, model, max_n_shifts = None):
    with open(shift_file, 'r') as file:
        data = json.load(file)
        df_shifts = pd.DataFrame(data)
    df_shifts = df_shifts[(df_shifts['outsourcing_cost_multiplier']==OC)&(df_shifts['regional_multiplier']==RM)&(df_shifts['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_shifts = df_shifts[df_shifts['model']==model]
    #partflex
    else:
        df_shifts = df_shifts[(df_shifts['model']==model)&(df_shifts['max_n_shifts']==max_n_shifts)]
    df_shifts.reset_index(drop = True, inplace=True)
    df_shifts = df_shifts[['region','shifts_start']]
    dict_shifts = {}
    for region in df_shifts['region'].unique().tolist():
        dict_shifts[region] = []
        dict_ = df_shifts[df_shifts['region']==region]['shifts_start'].tolist()[0]
        for shift_start in dict_.keys():
            dict_shifts[region].append(dict_[shift_start])
    return dict_shifts

In [17]:
def create_run_dictionary(base_file, OC, RM, GM):

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    #load all shifts
    dict_weekday = {}
    dict_weekend = {}
    dict_track = {}
    dict_opt = {}
    for model in ['fixed','flex','partflex']:
        if model == 'partflex':
            for max_n_shifts in range(2,5):
                dict_ = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts)
                dict_weekday[(model, max_n_shifts)] = dict_
                dict_ = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts)
                dict_weekend[(model, max_n_shifts)] = dict_
                dict_track[(model, max_n_shifts)] = []
                dict_opt[(model, max_n_shifts)] = 0
        else:
            dict_ = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts = None)
            dict_weekday[(model, None)] = dict_
            dict_ = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts = None)
            dict_weekend[(model, None)] = dict_
            dict_track[(model, None)] = []
            dict_opt[(model, None)] = 0

    #create a dictionary of distinct list of (model, max_n_shifts) that are the same

    list_models = [('fixed', None),('flex', None),('partflex',2),('partflex',3),('partflex',4)]
    list_ran = []
    for comb in list_models:
        list_ = list_models.copy()
        list_.remove(comb)
        for other_comb in list_:
            if (dict_weekday[comb]==dict_weekday[other_comb])&(dict_weekend[comb]==dict_weekend[other_comb])&(comb not in list_ran):
                dict_track[comb].append(other_comb)
                list_ran.append(other_comb)

    list_rmv = []
    for comb in dict_track.keys():
        ind_ = 0
        for comb_ in dict_track.keys():
            if comb in dict_track[comb_]:
                ind_ = 1

        if (dict_track[comb] == [])&(ind_ == 1):
            list_rmv.append(comb)
    for comb in list_rmv:
        del dict_track[comb]

    return dict_track

In [18]:
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

for base_file in ['paris_db=4.00']:
# for base_file in ['lyon_db=0.50']:

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    workforce_dict = {}

    city_pattern = r'(\w+)_db'
    db_pattern = r'db=(\d+\.\d+)'

    city_match = re.search(city_pattern, base_file)
    db_match = re.search(db_pattern, base_file)

    city = city_match.group(1) if city_match else None
    demand_baseline = float(db_match.group(1)) if db_match else None

    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    # for OC in [1.2]:

        #run only over distinct shift combinations (avoiding duplicates but still keeping track)

        dict_run = create_run_dictionary(base_file, OC, RM, GM)
        print(f'dict_run: {dict_run}')

        dict_str = {}
        for key in dict_run.keys():
            key_str = f'{key[0]}-{key[1]}'
            dict_str[key_str] = []
            for val in dict_run[key]:
                val_str = f'{val[0]}-{val[1]}'
                dict_str[key_str].append(val_str)

        for comb in dict_run.keys():
            model = comb[0]
            max_n_shifts = comb[1]

            if max_n_shifts == None:
                results_file = f"{base_file}_OC={OC}_model={model}.pkl"
            else:
                results_file = f"{base_file}_OC={OC}_model={model}_max_n_shifts={max_n_shifts}.pkl"

            if results_file not in os.listdir(r'../rerun_raw_results'):

                start_time = time.time()
                print(f'start_time: {datetime.fromtimestamp(start_time)}')
                dict_baseline, dict_results = \
                    run_roster_solver_objval_w_baseline(model, instance_file_weekday, 
                                                        shift_file_weekday, instance_file_weekend, 
                                                        shift_file_weekend, workforce_dict, 
                                                        OC, RM, GM, h_min, h_max, max_n_diff, 
                                                        max_n_shifts, expand_workforce_to_regions=True)
                
                dict_results['dict_run'] = dict_run
                dict_results['baseline'] = dict_baseline
                end_time = time.time()
                run_time = (end_time - start_time)
                dict_results['run_time'] = run_time

                # Save results
                with open(f'../rerun_raw_results/{results_file}', 'wb') as file:
                    pickle.dump(dict_results, file)
                print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, run_time: {run_time}')

                for subcomb in dict_run[comb]:
                    if subcomb[1] == None:
                        results_file_ = f"{base_file}_OC={OC}_model={subcomb[0]}.pkl"
                    else:
                        results_file_ = f"{base_file}_OC={OC}_model={subcomb[0]}_max_n_shifts={subcomb[1]}.pkl"

                    with open(f'../rerun_raw_results/{results_file_}', 'wb') as file:
                        pickle.dump(dict_results, file)

            else:
                for subcomb in dict_run[comb]:
                    dict_results = pd.read_pickle(f'../rerun_raw_results/{results_file}')
                    if subcomb[1] == None:
                        results_file_ = f"{base_file}_OC={OC}_model={subcomb[0]}.pkl"
                    else:
                        results_file_ = f"{base_file}_OC={OC}_model={subcomb[0]}_max_n_shifts={subcomb[1]}.pkl"

                    with open(f'../rerun_raw_results/{results_file_}', 'wb') as file:
                        pickle.dump(dict_results, file)


dict_run: {('fixed', None): [('partflex', 2)], ('flex', None): [('partflex', 3), ('partflex', 4)]}
start_time: 2024-06-04 18:57:55.363223


KeyboardInterrupt: 